# Chapter 5. BOW 기반의 문서 분류


## 5.1 20 뉴스그룹 데이터 준비 및 특성 추출

In [2]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

print('#Train set size: ', len(newsgroups_train.data))
print('#Test set size: ', len(newsgroups_test.data))
print('#Selected categories: ', newsgroups_train.target_names)
print('#Train labels: ', set(newsgroups_train.target))

#Train set size:  2034
#Test set size:  1353
#Selected categories:  ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
#Train labels:  {0, 1, 2, 3}


In [3]:
print('#Train set text samples: ', newsgroups_train.data[0])
print('#Train set label samples: ', newsgroups_train.target[0])
print('#Test set text samples: ', newsgroups_test.data[0])
print('#Test set label samples: ', newsgroups_test.target[0])

#Train set text samples:  Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
#Train set label samples:  1
#Test set text samples:  TRry the SKywatch project in  Arizona.
#Test set label samples:  2


In [4]:
X_train = newsgroups_train.data
y_train = newsgroups_train.target

X_test = newsgroups_test.data
y_test = newsgroups_test.target

from sklearn.feature_extraction.text import CountVectorizer

#min_df로 5개 미만의 문서에서 나타나는 단어는 특성에서 제외 및 max_df로 문서의 50%를 초과해 나타나는 단어 제외
cv = CountVectorizer(max_features=2000, min_df=5, max_df=0.5)

X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

print('#Train set dimension: ', X_train_cv.shape)
print('#Test set dimension: ', X_test_cv.shape)

#Train set dimension:  (2034, 2000)
#Test set dimension:  (1353, 2000)


In [5]:
for word, count in zip(cv.get_feature_names_out()[:100], X_train_cv[0].toarray()[0, :100]):
    print(word, ':', count, end=', ')

00 : 0, 000 : 0, 01 : 0, 04 : 0, 05 : 0, 10 : 0, 100 : 0, 1000 : 0, 11 : 0, 12 : 0, 128 : 0, 129 : 0, 13 : 0, 130 : 0, 14 : 0, 15 : 0, 16 : 0, 17 : 0, 18 : 0, 19 : 0, 1987 : 0, 1988 : 0, 1989 : 0, 1990 : 0, 1991 : 0, 1992 : 0, 1993 : 0, 20 : 0, 200 : 0, 202 : 0, 21 : 0, 22 : 0, 23 : 0, 24 : 0, 25 : 0, 256 : 0, 26 : 0, 27 : 0, 28 : 0, 2d : 0, 30 : 0, 300 : 0, 31 : 0, 32 : 0, 33 : 0, 34 : 0, 35 : 0, 39 : 0, 3d : 0, 40 : 0, 400 : 0, 42 : 0, 45 : 0, 50 : 0, 500 : 0, 60 : 0, 600 : 0, 65 : 0, 70 : 0, 75 : 0, 80 : 0, 800 : 0, 90 : 0, 900 : 0, 91 : 0, 92 : 0, 93 : 0, 95 : 0, _the : 0, ability : 0, able : 1, abortion : 0, about : 1, above : 0, absolute : 0, absolutely : 0, ac : 0, accept : 0, acceptable : 0, accepted : 0, access : 0, according : 0, account : 0, accurate : 0, across : 0, act : 0, action : 0, actions : 0, active : 0, activities : 0, activity : 0, acts : 0, actual : 0, actually : 0, ad : 0, add : 0, added : 0, addition : 0, additional : 0, address : 0, 

## 5.2 머신러닝과 문서 분류 과정에 대한 이해

## 5.3 나이브 베이즈 분류기를 이용한 문서 분류

In [6]:
from sklearn.naive_bayes import MultinomialNB

NB_clf = MultinomialNB()  #분류기 선언

#train set을 이용하여 분류기(classifier) 학습
NB_clf.fit(X_train_cv, y_train)

print('#Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train)))
print('#Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test)))

#Train set score: 0.824
#Test set score: 0.732


In [7]:
print('#First document and label in test data: ', X_test[0], y_test[0])
print('#Second document and label in test data: ', X_test[1], y_test[1])

pred = NB_clf.predict(X_test_cv[:2])

print('#Predicted labels: ', pred)
print('#Predicted categories: ', newsgroups_train.target_names[pred[0]], newsgroups_train.target_names[pred[1]])

#First document and label in test data:  TRry the SKywatch project in  Arizona. 2
#Second document and label in test data:  The Vatican library recently made a tour of the US.
 Can anyone help me in finding a FTP site where this collection is 
 available. 1
#Predicted labels:  [2 1]
#Predicted categories:  sci.space comp.graphics


In [8]:
#성능 개선 : TF-IDF 사용
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

NB_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf, y_test)))

Train set score: 0.862
Test set score: 0.741


In [9]:
import numpy as np

def top10_features(classifier, vectorizer, categories):
    feature_names = np.asarray(vectorizer.get_feature_names_out())
    for i, category in enumerate(categories):
        top10 = np.argsort(-classifier.coef_[i])[:10]
        print('%s: %s' % (category, ', '.join(feature_names[top10])))
        
top10_features(NB_clf, tfidf, newsgroups_train.target_names)

alt.atheism: you, not, are, be, this, have, as, what, they, if
comp.graphics: you, on, graphics, this, have, any, can, or, with, thanks
sci.space: space, on, you, be, was, this, as, they, have, are
talk.religion.misc: you, not, he, are, as, this, be, god, was, they


/Users/rose/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


## 5.4 로지스틱 회귀분석을 이용한 문서 분류

In [11]:
from sklearn.linear_model import LogisticRegression

LR_clf = LogisticRegression()
LR_clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test)))

#Train set score: 0.930
#Test set score: 0.734


### 릿지 회귀를 이용한 과적합 방지 : 계수가 커지지 않게 계수에 제약을 주는 방식

In [12]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier()
ridge_clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

#Train set score: 0.960
#Test set score: 0.735


In [15]:
#그리드 서치를 통해 최적의 하이퍼 파라미터 값 구하기
import numpy as np
from sklearn.model_selection import train_test_split

X_train_ridge, X_val_ridge, y_train_ridge, y_val_ridge = train_test_split(
    X_train_tfidf, y_train, test_size=0.2, random_state=42)

max_score = 0
max_alpha = 0
for alpha in np.arange(0.1, 10, 0.1):
    ridge_clf = RidgeClassifier(alpha=alpha)
    ridge_clf.fit(X_train_ridge, y_train_ridge)
    score = ridge_clf.score(X_val_ridge, y_val_ridge)
    if score > max_score:
        max_score = score
        max_alpha = alpha
        
print('#Max alpha {:.3f} at max validation score {:.3f}'.format(max_alpha, max_score))

#Max alpha 1.600 at max validation score 0.826


In [16]:
ridge_clf = RidgeClassifier(alpha=1.6)
ridge_clf.fit(X_train_tfidf, y_train)

print('#Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

#Train set score: 0.948
#Test set score: 0.739


In [17]:
top10_features(ridge_clf, tfidf, newsgroups_train.target_names)

alt.atheism: bobby, religion, atheism, atheists, motto, punishment, islam, deletion, islamic, satan
comp.graphics: graphics, computer, 3d, file, image, hi, 42, using, screen, looking
sci.space: space, orbit, nasa, spacecraft, moon, sci, launch, flight, funding, idea
talk.religion.misc: christian, christians, fbi, blood, order, jesus, objective, children, christ, hudson


### 라쏘 회귀를 이용한 특성 선택 : 계수가 0에 가까워지면 이를 완전히 0으로 바꾸는 방식(=특성의 수 줄이기)

In [20]:
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear', C=1)
lasso_clf.fit(X_train_tfidf, y_train)

print('#Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print(f'#Used features count: {np.sum(lasso_clf.coef_ != 0)} out of {X_train_tfidf.shape[1]}')

#Train set score: 0.819
#Test set score: 0.724
#Used features count: 437 out of 2000


In [21]:
top10_features(lasso_clf, tfidf, newsgroups_train.target_names)

alt.atheism: bobby, atheism, atheists, islam, religion, islamic, motto, atheist, satan, vice
comp.graphics: graphics, image, 3d, file, computer, hi, video, files, looking, sphere
sci.space: space, orbit, launch, nasa, spacecraft, flight, moon, dc, shuttle, solar
talk.religion.misc: fbi, christian, christians, christ, order, jesus, children, objective, context, blood


## 5.5 결정트리 등을 이용한 기타 문서 분류 방법

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

tree = DecisionTreeClassifier(random_state=7)
tree.fit(X_train_tfidf, y_train)
print('#Decision Tree train set score: {:.3f}'.format(tree.score(X_train_tfidf, y_train)))
print('#Decision Tree test set score: {:.3f}'.format(tree.score(X_test_tfidf, y_test)))

forest = RandomForestClassifier(random_state=7)
forest.fit(X_train_tfidf, y_train)
print('#Random Forest train set score: {:.3f}'.format(forest.score(X_train_tfidf, y_train)))
print('#Random Forest test set score: {:.3f}'.format(forest.score(X_test_tfidf, y_test)))

gradient = GradientBoostingClassifier(random_state=7)
gradient.fit(X_train_tfidf, y_train)
print('#Gradient Boosting train set score: {:.3f}'.format(gradient.score(X_train_tfidf, y_train)))
print('#Gradient Boosting test set score: {:.3f}'.format(gradient.score(X_test_tfidf, y_test)))

#Decision Tree train set score: 0.977
#Decision Tree test set score: 0.536
#Random Forest train set score: 0.977
#Random Forest test set score: 0.685
#Gradient Boosting train set score: 0.933
#Gradient Boosting test set score: 0.696


In [23]:
sorted_feature_importances = sorted(
    zip(tfidf.get_feature_names_out(), gradient.feature_importances_), key=lambda x: x[1], reverse=True)
for feature, value in sorted_feature_importances[:40]:
    print('%s: %.3f' % (feature, value), end=', ')

space: 0.126, graphics: 0.080, atheism: 0.024, thanks: 0.023, file: 0.021, orbit: 0.020, jesus: 0.018, god: 0.018, hi: 0.017, nasa: 0.015, image: 0.015, files: 0.014, christ: 0.010, moon: 0.010, bobby: 0.010, launch: 0.010, looking: 0.010, christian: 0.010, atheists: 0.009, christians: 0.009, fbi: 0.009, 3d: 0.008, you: 0.008, not: 0.008, islamic: 0.007, religion: 0.007, spacecraft: 0.007, flight: 0.007, computer: 0.007, islam: 0.007, ftp: 0.006, color: 0.006, software: 0.005, atheist: 0.005, card: 0.005, people: 0.005, koresh: 0.005, his: 0.005, kent: 0.004, sphere: 0.004, 

## 5.6 성능을 높이는 방법

In [26]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

RegTok = RegexpTokenizer("[\w']{3,}")
english_stops = set(stopwords.words('english'))

def tokenizer(text):
    tokens = RegTok.tokenize(text.lower())
    words = [word for word in tokens if (word not in english_stops) and len(word) > 2]
    features = (list(map(lambda token: PorterStemmer().stem(token), words)))
    return features

tfidf = TfidfVectorizer(tokenizer=tokenizer, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

LR_clf = LogisticRegression()
LR_clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test)))
print(len(LR_clf.coef_[0]))

#Train set score: 0.930
#Test set score: 0.751
2000


In [27]:
tfidf = TfidfVectorizer(tokenizer=tokenizer)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
print('#Train set dimension: ', X_train_tfidf.shape)
print('#Test set dimension: ', X_test_tfidf.shape)

ridge_clf = RidgeClassifier(alpha=2.4)
ridge_clf.fit(X_train_tfidf, y_train)
print('#Ridge Classifier train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('#Ridge Classifier test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

NB_clf = MultinomialNB(alpha=0.01)
NB_clf.fit(X_train_tfidf, y_train)
print('#Naive Bayse Classifier train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf, y_train)))
print('#Naive Bayse Classifier test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf, y_test)))

#Train set dimension:  (2034, 20085)
#Test set dimension:  (1353, 20085)
#Ridge Classifier train set score: 0.968
#Ridge Classifier test set score: 0.768
#Naive Bayse Classifier train set score: 0.971
#Naive Bayse Classifier test set score: 0.793


## 5.7 카운트 기반의 문제점과 N-gram을 이용한 보완

### unigram vs. bi-gram vs. tri-gram

In [29]:
#Unigram
tfidf = TfidfVectorizer(token_pattern="[a-zA-Z']{3,}",
                        decode_error='ignore',
                        lowercase=True,
                        stop_words=stopwords.words('english'),
                        max_df=0.5,
                        min_df=2)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
print('#Train set dimension: ', X_train_tfidf.shape)

ridge_clf = RidgeClassifier()
ridge_clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

#Train set dimension:  (2034, 11483)
#Train set score: 0.976
#Test set score: 0.766


In [30]:
#Bi-gram
tfidf = TfidfVectorizer(token_pattern="[a-zA-Z']{3,}",
                        decode_error='ignore',
                        lowercase=True,
                        stop_words=stopwords.words('english'),
                        ngram_range=(1, 2),
                        max_df=0.5,
                        min_df=2)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
print('#Train set dimension: ', X_train_tfidf.shape)


bigram_features = [f for f in tfidf.get_feature_names_out() if len(f.split()) > 1]
print('#Bi-gram samples: ', bigram_features[:10])

ridge_clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

#Train set dimension:  (2034, 26550)
#Bi-gram samples:  ["'cause can't", "'em better", "'expected errors'", "'karla' next", "'nodis' password", "'official doctrine", "'ok see", "'sci astro'", "'what's moonbase", 'aas american']
#Train set score: 0.976
#Test set score: 0.773


In [31]:
#Tri-gram
tfidf = TfidfVectorizer(token_pattern="[a-zA-Z']{3,}",
                        decode_error='ignore',
                        lowercase=True,
                        stop_words=stopwords.words('english'),
                        ngram_range=(1, 3),
                        max_df=0.5,
                        min_df=2)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
print('#Train set dimension: ', X_train_tfidf.shape)


trigram_features = [f for f in tfidf.get_feature_names_out() if len(f.split()) > 2]
print('#Tri-gram samples: ', bigram_features[:10])

ridge_clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

#Train set dimension:  (2034, 32943)
#Tri-gram samples:  ["'cause can't", "'em better", "'expected errors'", "'karla' next", "'nodis' password", "'official doctrine", "'ok see", "'sci astro'", "'what's moonbase", 'aas american']
#Train set score: 0.976
#Test set score: 0.775


## 5.8 한국어 문서의 분류

In [36]:
import pandas as pd
df = pd.read_csv('./data/daum_movie_review.csv')
df.head(5)

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [38]:
df.title.value_counts()

신과함께      4947
택시운전사     2322
인피니티 워    2042
범죄도시      1939
곤지암       1547
라라랜드      1150
코코         778
Name: title, dtype: int64

In [39]:
from sklearn.model_selection import train_test_split

#비율을 지정하지 않으면 75:25로 분할
X_train, X_test, y_train, y_test = train_test_split(df.review, df.title, random_state=0)

print('#Train set size: ', len(X_train))
print('#Test set size: ', len(X_test))

#Train set size:  11043
#Test set size:  3682


In [40]:
from konlpy.tag import Okt
okt = Okt()

print(okt.morphs(X_train[1]))
print(okt.nouns(X_train[1]))

['몰입', '할수밖에', '없다', '.', '어렵게', '생각', '할', '필요없다', '.', '내', '가', '전투', '에', '참여', '한', '듯', '손', '에', '땀', '이남', '.']
['몰입', '생각', '내', '전투', '참여', '듯', '손', '땀', '이남']


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)  #충분한 학습을 위해 max_iter를 1000으로 설정, 기본은 100
clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.756
#Test set score: 0.694


In [42]:
print('실제 영화 제목, 예측한 제목, 리뷰')
for content in zip(y_test[:10], clf.predict(X_test_tfidf[:10]), X_test[:10]):
    print(content)

실제 영화 제목, 예측한 제목, 리뷰
('범죄도시', '신과함께', '오랜만에 잼나는 영화 봤습니다.  다음에 더 재미있는 영화 기대하겠습니다.')
('범죄도시', '범죄도시', '조연들이 눈에 박힌다. 간만에 집중 ㅎ')
('코코', '코코', '대감동을 선사. 인사이드 아웃을 잇는 픽사의 감동스토리. 신과함께의 멕시코판이라고나할까요??')
('신과함께', '신과함께', '돈이 안아까웠던 영화ᆞᆞ  정말 좋았다')
('신과함께', '신과함께', '역시 김용화감독이 영화는 잘 만들어요. 이제 VFX 제작 부문도 헐리우드 수준 이상입니다.')
('택시운전사', '택시운전사', '민주화를 위해 힘써주신 분들께 감사하는 마음으로 살아야겠다.')
('신과함께', '신과함께', '잠만 자다 왔음')
('신과함께', '신과함께', '오랜만에 잼있고 좋은 영화를 봤다')
('범죄도시', '신과함께', '잼남')
('범죄도시', '인피니티 워', '대박~~')


### 성능을 개선하기 위한 노력

In [43]:
#명사 대신 모든 형태소를 사용
tfidf = TfidfVectorizer(tokenizer=okt.morphs, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.777
#Test set score: 0.695


In [44]:
#전체를 다 사용하는 대신, 명사, 동사, 형용사를 사용
def twit_tokenizer(text):
    target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in okt.pos(text, norm=True, stem=True):
        if tag in target_tags:
            result.append(word)
    return result

tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.784
#Test set score: 0.712


In [46]:
#모든 형태소를 다 사용하고 품사를 알 수 있도록 설정
def twit_tokenizer2(text):
    result = []
    for word, tag in okt.pos(text, norm=True, stem=True):
        result.append('/'.join([word, tag])) #단어의 품사를 구분할 수 있도록 함
    return result

print(twit_tokenizer2(X_train[1]))

tfidf = TfidfVectorizer(tokenizer=twit_tokenizer2, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

['몰입/Noun', '하다/Verb', '없다/Adjective', './Punctuation', '어렵다/Adjective', '생각/Noun', '하다/Verb', '필요없다/Adjective', './Punctuation', '내/Noun', '가/Josa', '전투/Noun', '에/Josa', '참여/Noun', '한/Determiner', '듯/Noun', '손/Noun', '에/Josa', '땀/Noun', '이남/Noun', './Punctuation']
#Train set score: 0.789
#Test set score: 0.718


In [47]:
#그리드 서치
import numpy as np
from sklearn.model_selection import train_test_split

X_train_ridge, X_val_ridge, y_train_ridge, y_val_ridge = train_test_split(
    X_train_tfidf, y_train, test_size=0.2, random_state=42)

max_score = 0
max_alpha = 0
for alpha in np.arange(0.1, 10, 0.1):
    ridge_clf = RidgeClassifier(alpha=alpha)
    ridge_clf.fit(X_train_ridge, y_train_ridge)
    score = ridge_clf.score(X_val_ridge, y_val_ridge)
    if score > max_score:
        max_score = score
        max_alpha = alpha
print('#Max alpha {:.3f} at max validation score {:.3f}'.format(max_alpha, max_score))

#Max alpha 1.600 at max validation score 0.727


In [48]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier(alpha=1.6)
ridge_clf.fit(X_train_tfidf, y_train)
print('#Ridge Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('#Ridge Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

from sklearn.linear_model import LogisticRegression
import numpy as np
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear', C=0.5)
lasso_clf.fit(X_train_tfidf, y_train)
print('#Lasso Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('#Lasso Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print('#Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1])

#Ridge Train set score: 0.807
#Ridge Test set score: 0.726
#Lasso Train set score: 0.703
#Lasso Test set score: 0.696
#Used features count: 957 out of 2000


In [49]:
from sklearn.naive_bayes import MultinomialNB

NB_clf = MultinomialNB(alpha=0.1)
NB_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf, y_test)))

Train set score: 0.768
Test set score: 0.704
